# Эксперименты с финальной версией
В final_clean_version была реализована ВРОДЕ КАК правильная структура с разбиением на KFold и обучением (валидацией) на ряде фолдов и с соответствующей оценкой. Но получили ухудшение оценок сильное и тут, и в kaggle. Тут попробуем повторить все тоже самое, но без логарифмов и без доп. фич, чтобы попробовать на Kfold-ах получить приемлемые потери и скор = 12800 в kaggle (самый лучший из improved_v6)

In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import pandas as pd
import seaborn as sns
import warnings

from catboost import CatBoostRegressor
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from utils.data_manager import DataManager
from utils.model_manager import ModelManager

In [2]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [3]:
dm = DataManager()
mm = ModelManager()

# Отключаем автологгирование, чтобы использовать ручное
mlflow.sklearn.autolog(disable=True)
warnings.filterwarnings("ignore", module="mlflow")  # Игнорируем предупреждения MLflow


In [4]:
RANDOM_STATE = 42

## 1. Загрузка данных

In [5]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

## 2. Предобработка данных 

In [6]:
# Определение колонок для удаления
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)

In [7]:
# Разделение на X / y
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)
X_test = test_data.copy()
print(X_test.shape)

(1460, 80) (1460,)
(1459, 80)


In [8]:
X.drop(columns=bad_columns, inplace=True)
X_test.drop(columns=bad_columns, inplace=True)

In [9]:
def make_feature_eng_great_again(train_X_in, test_X_in):
    """Хелпер, который создает фичи, логарифмирует и выравнивает колонки."""
    # Работаем с копиями, чтобы не изменять оригинальные X, X_test вне функции
    train_X = train_X_in.copy()
    test_X = test_X_in.copy()

    # Словарь качественных признаков
    quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}

    def create_features(df):
        # Interactions (с проверкой на наличие колонок)
        if 'Neighborhood' in df.columns and 'MSZoning' in df.columns:
            df['Neighborhood_Zoning'] = df['Neighborhood'].astype(str) + '_' + df['MSZoning'].astype(str)
        if 'SaleType' in df.columns and 'SaleCondition' in df.columns:
            df['SaleType_Condition'] = df['SaleType'].astype(str) + '_' + df['SaleCondition'].astype(str)

        # Quality Score
        df['TotalQualScore'] = 0
        quality_cols = ['ExterQual', 'KitchenQual', 'BsmtQual', 'HeatingQC', 'GarageQual', 'FireplaceQu']
        for col in quality_cols:
            if col in df.columns:
                 df['TotalQualScore'] += df[col].map(quality_dict).fillna(0)

        # Porch/Deck Area and Flags
        df['PorchDeckArea'] = 0
        porch_cols = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
        for col in porch_cols:
             if col in df.columns:
                df['PorchDeckArea'] += df[col].fillna(0)

        if 'Fireplaces' in df.columns:
            df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
        if 'GarageType' in df.columns:
            df['HasGarage'] = (~df['GarageType'].isna()).astype(int)
        if 'Fence' in df.columns:
            df['HasFence'] = (~df['Fence'].isna()).astype(int)
        df['HasPorchDeck'] = (df['PorchDeckArea'] > 0).astype(int)

        return df

    def log_features(df, cols_to_log_list):  # Принимает СПИСОК колонок
        print(f"Applying log1p to: {cols_to_log_list}")
        for col_name in cols_to_log_list:
            if col_name in df.columns:
                # Добавим проверку на отрицательные значения перед логарифмированием
                if (df[col_name] < 0).any():
                     print(f"Warning: Column {col_name} contains negative values. Skipping log1p.")
                else:
                    df[col_name] = np.log1p(df[col_name])
            else:
                print(f"Warning: Column {col_name} not found in DF during log transform.")
        return df

    # 1. Создаем фичи
    train_X = create_features(train_X)
    test_X = create_features(test_X)
    print("Features created.")

    # 2. Определяем колонки для логарифмирования (ТОЛЬКО по трейну)
    numeric_cols = train_X.select_dtypes(include=np.number).columns
    skew_values = train_X[numeric_cols].skew()
    # Используем .index.tolist() чтобы получить список имен
    cols_to_log_list = skew_values[skew_values > 1].index.tolist()
    print(f"Columns identified for logging: {cols_to_log_list}")

    # 3. Логарифмируем (используя ОДИН и тот же список)
    train_X = log_features(train_X, cols_to_log_list)
    test_X = log_features(test_X, cols_to_log_list)
    print("Log transform applied.")

    # 4. Согласуем и сортируем колонки ПОСЛЕ всех манипуляций
    final_feature_cols = sorted(train_X.columns.tolist()) # Сортируем для стабильности
    train_X = train_X[final_feature_cols]
    test_X = test_X.reindex(columns=final_feature_cols, fill_value=0)
    print("Columns aligned and sorted.")

    return train_X, test_X

In [10]:
# Вызываем функцию с правильными данными (X, X_test)
X, X_test = make_feature_eng_great_again(X, X_test)

print("\nProcessing complete. Final shapes:")
print(f"X_processed: {X.shape}")
print(f"X_test_processed: {X_test.shape}")
print("\nExample processed X:")
print(X.head())

Features created.
Columns identified for logging: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PorchDeckArea', 'HasFence']
Applying log1p to: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PorchDeckArea', 'HasFence']
Applying log1p to: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PorchDeckArea', 'HasFence']
Log transform applied.
Columns aligned and sorted.

Pr

In [11]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

### Аналитика проблемных образцов, дающих очень плохие ошибки в фолдах

In [12]:
# # Проблемные данные с жуткими ошибками из 3 Фолда
# # 523           60       RL        130.0    40094   Pave   NaN    AllPub   
# # 1298          60       RL        313.0    63887   Pave   NaN    AllPub   
# # 1169          60       RL        118.0    35760   Pave   NaN    AllPub   
# problematic_indices = [523, 1298, 1169, 1324, 440] # Индексы из твоего отчета об ошибках
# 
# # Посмотреть всю информацию по этим строкам (домам)
# print("Original data for problematic houses:")
# print(train_data.loc[problematic_indices])
# 
# # Посмотреть только SalePrice для проверки
# print("\nSalePrice for problematic houses:")
# print(train_data.loc[problematic_indices, 'SalePrice'])


In [13]:
# # Предполагаем, что X - это твои обработанные фичи (DataFrame)
# problematic_indices = [523, 1298, 1169, 1324, 440] # Как определили ранее
# problematic_indices = [523, 1298]  # конкретно эти дома модель сильно переоценила
# 
# # Разделяем индексы
# all_indices = X.index # Все индексы в твоем трейн-сете X
# rest_indices = all_indices.difference(problematic_indices)
# 
# # Сравниваем числовые признаки
# print("--- Описательные статистики для ПРОБЛЕМНЫХ домов ---")
# print(X.loc[problematic_indices, numeric_columns].describe()) # numeric_columns - список числовых колонок
# 
# print("\n--- Описательные статистики для ОСТАЛЬНЫХ домов ---")
# print(X.loc[rest_indices, numeric_columns].describe())
# 
# # Сравниваем категориальные признаки (пример для одной колонки)
# cat_col_to_compare = 'Neighborhood'
# print(f"\n--- Распределение '{cat_col_to_compare}' для ПРОБЛЕМНЫХ домов ---")
# print(X.loc[problematic_indices, cat_col_to_compare].value_counts(normalize=True))
# 
# print(f"\n--- Распределение '{cat_col_to_compare}' для ОСТАЛЬНЫХ домов ---")
# print(X.loc[rest_indices, cat_col_to_compare].value_counts(normalize=True))

In [14]:
# # "--- Описательные статистики для ПРОБЛЕМНЫХ домов ---"
# """
# Top 5 errors for this fold:
#       Actual      Predicted          Error
# 523   184750  515922.462045  331172.462045
# 1298  160000  428474.481532  268474.481532
# 1169  625000  467748.048480  157251.951520
# 1324  147000  281692.643270  134692.643270
# 440   555000  421663.071273  133336.928727
# """
# X.loc[problematic_indices, numeric_columns]

In [15]:
# # Попробуем найти схожие по стоимости дома и сравнить показатели
# target_price = 184_000
# tolerance = 10_000  # Максимально допустимая разница
# 
# rest_df = train_data.loc[rest_indices]
# same_houses = rest_df[abs(rest_df['SalePrice'] - target_price) <= tolerance]
# same_houses

In [16]:
# # -- Описательные статистики для ОСТАЛЬНЫХ домов ---
# X.loc[rest_indices, numeric_columns].describe()

In [17]:
# # Пример: Scatter plot GrLivArea vs SalePrice
# plt.figure(figsize=(10, 6))
# # Сначала рисуем все точки
# sns.scatterplot(x=X.loc[rest_indices, 'GrLivArea'], y=y.loc[rest_indices], label='Остальные', alpha=0.5)
# # Затем выделяем проблемные
# sns.scatterplot(x=X.loc[problematic_indices, 'GrLivArea'], y=y.loc[problematic_indices], color='red', s=100, label='Проблемные (Fold 3)', marker='X')
# plt.title('GrLivArea vs SalePrice')
# plt.xlabel('GrLivArea')
# plt.ylabel('SalePrice')
# plt.legend()
# plt.grid(True)
# plt.show()
# 
# # Пример: Box plot для OverallQual
# plt.figure(figsize=(8, 5))
# # Создаем временный DataFrame для удобства
# plot_df = X[['OverallQual']].copy()
# plot_df['Group'] = np.where(plot_df.index.isin(problematic_indices), 'Проблемные', 'Остальные')
# sns.boxplot(x='Group', y='OverallQual', data=plot_df)
# plt.title('Сравнение OverallQual')
# plt.show()

## 3. Обучаем модель с CV и корректируя данные

In [18]:
# --- Логарифмирование целевой переменной ---
y_log = np.log1p(y)

#### Нормализация данных через ColumnTransformer и Pipeline
В данном кейсе мы реализуем Заполнение числовых пропусков с помощью модели (Predictive imputation). Т.е. то, что пропущено в числовых признаках - будем заполнять не медианой или средним, а будем обучать модель, которая будет предсказывать пропуски (IterativeImputer + RandomForestRegressor)

In [19]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# Пайплайн для числовых признаков (итеративное заполнение)
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
        max_iter=10,
        random_state=RANDOM_STATE
    )),  # Дает примерно +100 прирост качества vs mean/median
    # ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())  # разницы не дает 
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- Объединяем препроцессоры ---
preprocessor = ColumnTransformer(
    transformers=[
        # Применяем к исходным числовым колонкам
        ('num', numeric_transformer, numeric_columns),
        # Применяем к исходным категориальным колонкам
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
    verbose_feature_names_out=False  # Чтобы имена колонок не менялись на 'num__colname' и т.д.
)

default_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 500  # ?
}
catboost_params = {
    'iterations': 3000,  # Увеличиваем запас итераций, учитывая early_stopping
    'learning_rate': 0.03,
    'depth': 6,
    'loss_function': 'RMSE', # Оптимизируем RMSE на логарифмированной шкале
    'verbose': 0,
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 100 # Включаем early stopping
}
optuna_best_params = {
    'iterations': 2279,
    'depth': 5,
    'learning_rate': 0.01489174151229139,
    'l2_leaf_reg': 2.3495954457881623,
    'bagging_temperature': 0.9217322652060611,
    'loss_function': 'RMSE',
    'random_seed': 42,
    'verbose': 0,
    'early_stopping_rounds': 500
}
optuna_best_params_v2 = {
    'iterations': 1775,
    'depth': 5,
    'learning_rate': 0.06039828474626542,
    'l2_leaf_reg': 2.132889759843395,
    'bagging_temperature': 0.9470181666458671,
    'subsample': 0.833424506156886,
    'colsample_bylevel': 0.9622754820457864,
    'loss_function': 'RMSE',
    'random_seed': 42,
    'verbose': 0,
    'early_stopping_rounds': 500
}
catboost_params = optuna_best_params_v2

# Включить Early Stopping очень рекомендуется в параметрах. Это позволит модели на каждом фолде останавливаться тогда, когда метрика на валидационной части этого фолда (X_val, y_val_log) перестает улучшаться. Это самый надежный способ подобрать оптимальное число итераций для каждого фолда и избежать переобучения

# --- Финальный пайплайн ---
# Мы будем использовать preprocessor и модель отдельно в цикле CV
# для корректной работы early stopping с пайплайном sklearn. Поэтому нам финальный пайплайн - не нужен.
# final_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
#     ('model', CatBoostRegressor(**catboost_params))
# ])

In [20]:
"""
Пояснения к тому, что тут происходит:

Тут мы ОЦЕНИВАЕМ и ТЕСТИРУЕМ подобранные фичи и гиперпараметры. В цикле бежим по N_FOLD-ам, бьем данные тренировочные в соотношении 80% на обучение, 20% на валидацию. Обучаем модель на трейне, валидируем на валидации. Смотрим на ошибки. Если по итогу нас все устраивает, мы дальше идем и обучаем модель с нуля на всех данных со всеми гиперпараметрами и фичами (да еще можно вычленить из цикла итерацию самую лучшую для нужного количества итераций обучения финальной модели),

oof_predictions — это массив, который в итоге будет содержать предсказания для каждого объекта из исходного тренировочного набора (X). Важно, что предсказание для конкретного объекта (например, дома №100) делается той моделью (из цикла CV), которая обучалась без этого объекта. Надежная оценка качества: OOF-предикты позволяют посчитать метрику качества (например, oof_rmse) на всем тренировочном наборе, при этом каждое предсказание было сделано "честно" (модель не видела этот объект при обучении). 
Эта оценка часто бывает более надежной, чем простое усреднение метрик по фолдам (mean_cv_rmse), так как она считается на полном наборе данных один раз. 
Можно сравнить oof_predictions с реальными значениями y_log (или y), чтобы понять, на каких объектах модель ошибается сильнее всего.
Стэкинг/Блендинг: OOF-предикты часто используются как новые признаки для обучения модели второго уровня (мета-модели) в ансамблях (стэкинг).
"""

# --- Кросс-Валидация ---
N_FOLDS = 5  # Например, 5 или 10
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

oof_predictions_log = np.zeros(X.shape[0])  # Для хранения out-of-fold предсказаний
fold_rmses = []
fold_best_iterations = []  # Будем сохранять лучшие итерации

mlflow.set_experiment("Final Version With KFold CV WITH Log + With IterativeImputer + WITH Features")
with mlflow.start_run() as run:  # Сохраняем run для логирования артефактов
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)): 
        print(f"--- Fold {fold+1}/{N_FOLDS} ---")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train_log, y_val_log = y_log.iloc[train_idx], y_log.iloc[val_idx]

        # 1. Обучаем препроцессор ТОЛЬКО на трейне текущего фолда
        preprocessor.fit(X_train)

        # 2. Трансформируем трейн и валидацию
        X_train_prep = preprocessor.transform(X_train)
        X_val_prep = preprocessor.transform(X_val)

        # 3. Обучаем модель с early stopping
        model = CatBoostRegressor(**catboost_params)
        model.fit(X_train_prep, y_train_log,
                  eval_set=[(X_val_prep, y_val_log)],
                  verbose=0,  # Отключаем вывод, 100 - если хотим видеть обучение 
                 )

        # Сохраняем лучшую итерацию
        best_iter = model.get_best_iteration()
        fold_best_iterations.append(best_iter)
        print(f"Best iteration for fold {fold+1}: {best_iter}")

        # 4. Предсказания на валидации
        val_preds_log = model.predict(X_val_prep)
        oof_predictions_log[val_idx] = val_preds_log

        # 5. Оценка на фолде (в исходной шкале)
        val_preds_orig = np.expm1(val_preds_log)
        y_val_orig = np.expm1(y_val_log)
        fold_rmse = np.sqrt(mean_squared_error(y_val_orig, val_preds_orig))
        print(f"Fold {fold+1} RMSE: {fold_rmse}")
        fold_rmses.append(fold_rmse)
        mlflow.log_metric(f"fold_{fold+1}_rmse", fold_rmse, step=fold+1)
        mlflow.log_metric(f"fold_{fold+1}_best_iter", best_iter, step=fold+1)
    
    # --- Итоговая оценка CV ---
    mean_cv_rmse = np.mean(fold_rmses)
    std_cv_rmse = np.std(fold_rmses)
    oof_rmse = np.sqrt(mean_squared_error(np.expm1(y_log), np.expm1(oof_predictions_log)))

    print(f"\nMean CV RMSE: {mean_cv_rmse:.4f} +/- {std_cv_rmse:.4f}")
    print(f"OOF RMSE: {oof_rmse:.4f}")
    print(f"Mean best iteration: {np.mean(fold_best_iterations):.0f}")

    # Логгирование итоговых метрик вручную
    mlflow.log_metric("mean_cv_rmse", mean_cv_rmse)
    mlflow.log_metric("std_cv_rmse", std_cv_rmse)
    mlflow.log_metric("oof_rmse", oof_rmse)
    mlflow.log_metric("mean_best_iteration", np.mean(fold_best_iterations))

--- Fold 1/5 ---
Best iteration for fold 1: 450
Fold 1 RMSE: 24679.24964434139
--- Fold 2/5 ---
Best iteration for fold 2: 683
Fold 2 RMSE: 23542.51539513202
--- Fold 3/5 ---
Best iteration for fold 3: 78
Fold 3 RMSE: 34876.44034313925
--- Fold 4/5 ---
Best iteration for fold 4: 1654
Fold 4 RMSE: 24917.10796261652
--- Fold 5/5 ---
Best iteration for fold 5: 623
Fold 5 RMSE: 17868.597663028373

Mean CV RMSE: 25176.7822 +/- 5485.9589
OOF RMSE: 25767.5398
Mean best iteration: 698


In [21]:
# К чему стремимся
# --- Fold 1/5 ---
# Best iteration for fold 1: 450
# Fold 1 RMSE: 24679.24964434139
# --- Fold 2/5 ---
# Best iteration for fold 2: 683
# Fold 2 RMSE: 23542.51539513202
# --- Fold 3/5 ---
# Best iteration for fold 3: 78
# Fold 3 RMSE: 34876.44034313925
# --- Fold 4/5 ---
# Best iteration for fold 4: 1654
# Fold 4 RMSE: 24917.10796261652
# --- Fold 5/5 ---
# Best iteration for fold 5: 623
# Fold 5 RMSE: 17868.597663028373
# 
# Mean CV RMSE: 25176.7822 +/- 5485.9589
# OOF RMSE: 25767.5398
# Mean best iteration: 698


In [26]:
# На прошлом шаге мы сделали KFOLD обучение, если довольны метриками, то получаем лучшую итерацию и обучаем финальную модель на ВСЕХ данных.

final_iterations = catboost_params['iterations']  # optuna нашла уже лучшие параметры
print(f"\nTraining final model on all data with {final_iterations} iterations...")

final_catboost_params = catboost_params.copy()
final_catboost_params['iterations'] = final_iterations
final_catboost_params.pop('early_stopping_rounds', None)  # Убираем early stopping для финального обучения
print(f"Финальные параметры обычения модели: {final_catboost_params}")

final_model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', CatBoostRegressor(**final_catboost_params))
])

final_model_pipeline.fit(X, y_log)
print("Final model trained.")


Training final model on all data with 1775 iterations...
Финальные параметры обычения модели: {'iterations': 1775, 'depth': 5, 'learning_rate': 0.06039828474626542, 'l2_leaf_reg': 2.132889759843395, 'bagging_temperature': 0.9470181666458671, 'subsample': 0.833424506156886, 'colsample_bylevel': 0.9622754820457864, 'loss_function': 'RMSE', 'random_seed': 42, 'verbose': 0}
Final model trained.


In [27]:
# Предсказание на тест данных для сабмишна
final_test_pred_log_single_model = final_model_pipeline.predict(X_test)
final_test_pred_single_model = np.expm1(final_test_pred_log_single_model)
submission_single = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': final_test_pred_single_model})
submission_single.to_csv('submission_final_kfold_with_log_v21.csv', index=False)
submission_single.head()

,Id,SalePrice
0,1461,128897.222323
1,1462,162091.781886
2,1463,190982.438750
3,1464,199984.837778
4,1465,184856.317464


In [28]:
# ===== Попробуем сделать Deep Ensemble =====
# Попытаемся сделать ансамбль моделей одной и той же архитектуры, но с разными Random State-ами, что по теории должно дать прирост точности. Можно это называть еще "Seed Ensembling".

N_SEEDS = 10  # Количество моделей в ансамбле
all_test_preds_log = []

# Шаги 3 и 4: Обучаем препроцессор и трансформируем данные
print("Fitting preprocessor on all X...")
preprocessor.fit(X)
print("Transforming X...")
X_prep_full = preprocessor.transform(X)
print("Transforming X_test...")
X_test_prep = preprocessor.transform(X_test)
print("Preprocessing complete.")

# Шаги 1 и 2 уже сделаны - best_params и final_iterations известны
final_catboost_params_ensemble = catboost_params.copy()
final_catboost_params_ensemble['iterations'] = final_iterations
final_catboost_params_ensemble.pop('early_stopping_rounds', None)
final_catboost_params_ensemble.pop('random_seed', None)
print(f"\nBest params for ensemble: {final_catboost_params_ensemble}")

# Шаг 5: Обучаем N моделей
print(f"\nTraining {N_SEEDS} models for ensemble...")
for i in range(N_SEEDS):
    current_seed = RANDOM_STATE + i
    print(f"Training model {i+1}/{N_SEEDS} with seed {current_seed}...")
    model = CatBoostRegressor(**final_catboost_params_ensemble, random_seed=current_seed)
    model.fit(X_prep_full, y_log) # Обучаем на всех X_prep_full

    # Шаг 6: Предсказание на тесте
    test_preds_log = model.predict(X_test_prep)
    all_test_preds_log.append(test_preds_log)
    print(f"Model {i+1} finished.")

# Шаг 7: Усреднение предсказаний
print("\nAveraging predictions...")
averaged_test_preds_log = np.mean(all_test_preds_log, axis=0)

# Шаг 8: Преобразование обратно
averaged_test_preds_orig = np.expm1(averaged_test_preds_log)
# Обработка возможных отрицательных значений
averaged_test_preds_orig[averaged_test_preds_orig < 0] = 0

# Шаг 9: Создание сабмишна
print("\nCreating submission file...")
submission_ensemble = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': averaged_test_preds_orig})
submission_ensemble.to_csv('submission_seed_ensemble_v1.csv', index=False)
print("Submission file created: submission_seed_ensemble_v1.csv")
print(submission_ensemble.head())

Fitting preprocessor on all X...
Transforming X...
Transforming X_test...
Preprocessing complete.

Best params for ensemble: {'iterations': 1775, 'depth': 5, 'learning_rate': 0.06039828474626542, 'l2_leaf_reg': 2.132889759843395, 'bagging_temperature': 0.9470181666458671, 'subsample': 0.833424506156886, 'colsample_bylevel': 0.9622754820457864, 'loss_function': 'RMSE', 'verbose': 0}

Training 10 models for ensemble...
Training model 1/10 with seed 42...
Model 1 finished.
Training model 2/10 with seed 43...
Model 2 finished.
Training model 3/10 with seed 44...
Model 3 finished.
Training model 4/10 with seed 45...
Model 4 finished.
Training model 5/10 with seed 46...
Model 5 finished.
Training model 6/10 with seed 47...
Model 6 finished.
Training model 7/10 with seed 48...
Model 7 finished.
Training model 8/10 with seed 49...
Model 8 finished.
Training model 9/10 with seed 50...
Model 9 finished.
Training model 10/10 with seed 51...
Model 10 finished.

Averaging predictions...

Creating s

# ИТОГО
Выше мы получили корректный код с K-FOLD валидацией, логарифмированием целевой переменной и предсказанием. Теперь его надо оптимизировать. Попробуем optuna.

In [27]:
import optuna

# Определяем KFold один раз вне objective
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# --- Функция цели для Optuna ---
def objective(trial):
    """Функция, которую Optuna будет минимизировать."""

    # 1. Определяем подбираемые параметры CatBoost
    catboost_params = {
        'iterations': trial.suggest_int('iterations', 1000, 4000),  # Оставляем диапазон, т.к. есть early stopping
        'depth': trial.suggest_int('depth', 4, 8),  # Слегка ограничиваем максимальную глубину
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.07, log=True),  # Сужаем диапазон LR
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0, log=True),  # Хороший диапазон для L2
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0), # Оставляем
        # Доля объектов, используемых для построения каждого дерева. Значения меньше 1.0 могут сделать модель более устойчивой к выбросам в данных.
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),  # --- Добавлено для робастности ---
        # Доля признаков, используемых на каждом уровне дерева. Тоже помогает бороться с переобучением.
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),  # --- Добавлено для робастности ---
        # --- Фиксированные параметры ---
        'loss_function': 'RMSE',
        'verbose': 0,
        'random_seed': RANDOM_STATE,
        'early_stopping_rounds': 500  # --- Увеличиваем терпение ---
    }

    fold_rmses = [] # Список для хранения RMSE по фолдам для текущего trial

    # 2. Запускаем цикл KFold CV
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y_log)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train_log, y_val_log = y_log.iloc[train_idx], y_log.iloc[val_idx]

        try:
            # 3. Препроцессинг
            # Копирование preprocessor не нужно, т.к. fit перезаписывает состояние
            preprocessor.fit(X_train)
            X_train_prep = preprocessor.transform(X_train)
            X_val_prep = preprocessor.transform(X_val)

            # 4. Обучение модели CatBoost
            model = CatBoostRegressor(**catboost_params)
            model.fit(X_train_prep, y_train_log,
                      eval_set=[(X_val_prep, y_val_log)],
                      verbose=0
                     )

            # 5. Предсказание и оценка
            val_preds_log = model.predict(X_val_prep)
            val_preds_orig = np.expm1(val_preds_log)
            y_val_orig = np.expm1(y_val_log)
            val_preds_orig[val_preds_orig < 0] = 0 # Обработка на всякий случай
            fold_rmse = np.sqrt(mean_squared_error(y_val_orig, val_preds_orig))

            # Проверка на NaN/inf перед добавлением
            if np.isnan(fold_rmse) or np.isinf(fold_rmse):
                 print(f"Warning: Invalid RMSE ({fold_rmse}) encountered in fold {fold+1}, trial {trial.number}. Pruning.")
                 # Можно сразу прекратить trial, если RMSE некорректен
                 raise optuna.exceptions.TrialPruned()

            fold_rmses.append(fold_rmse)

            # 6. Отчет для Optuna Pruning
            trial.report(fold_rmse, step=fold)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        except optuna.exceptions.TrialPruned:
             # Если trial был остановлен прунером, просто выходим
             raise
        except Exception as e:
             print(f"Error during fold {fold+1} for trial {trial.number}: {e}")
             # В случае другой ошибки, возвращаем 'inf'
             return float('inf')

    # 7. Возвращаем средний RMSE
    if not fold_rmses:  # Если все фолды вызвали ошибку (маловероятно)
        return float('inf')

    mean_rmse = np.mean(fold_rmses)
    std_rmse = np.std(fold_rmses)  # Считаем std dev для информации

    # Сохраняем std dev как атрибут, чтобы видеть его в результатах
    trial.set_user_attr("std_cv_rmse", std_rmse)

    print(f"Trial {trial.number} finished with Mean CV RMSE: {mean_rmse:.4f} +/- {std_rmse:.4f}")
    return mean_rmse

# --- Запуск исследования Optuna ---
# Pruner MedianPruner с n_warmup_steps=2 - хороший выбор по умолчанию
# "Pruning" (обрезка, отсечение) — это механизм в Optuna, который позволяет досрочно останавливать "бесперспективные" испытания (trials).
# Вместо того чтобы запускать каждую комбинацию гиперпараметров на всех 5 фолдах, Optuna после нескольких фолдов (в нашем случае после 3-го фолда, т.к. n_warmup_steps=2) смотрит на промежуточный результат (RMSE).
# Если этот промежуточный результат значительно хуже, чем у других, уже завершенных испытаний на том же шаге, Optuna решает, что нет смысла тратить время на оставшиеся фолды для этой комбинации параметров, и "обрезает" испытание.
# study = optuna.create_study(direction='minimize',
#                             pruner=optuna.pruners.MedianPruner(n_warmup_steps=2))
# Выбираем менее агрессивный прунер: Вместо MedianPruner (отсекает все, что хуже 50-го перцентиля) - PercentilePruner с меньшим порогом, например, 75-й перцентиль (проним лишь те испытания, которые хуже 25% самых плохих прошлых). Потому что у нас на 3 фолде очень большие ошибки почему-то всегда.
study = optuna.create_study(direction='minimize',
                            pruner=optuna.pruners.PercentilePruner(percentile=75, n_warmup_steps=3))

# Запускаем оптимизацию
N_TRIALS = 200 # Можно начать со 100, потом увеличить если нужно
study.optimize(objective, n_trials=N_TRIALS, n_jobs=1)  # n_jobs=1 - безопаснее

# --- Результаты ---
# (Код вывода результатов остается таким же)
# ...

# Сохраняем лучшие параметры
best_params = study.best_trial.params
best_params['loss_function'] = 'RMSE'
best_params['random_seed'] = RANDOM_STATE
best_params['verbose'] = 0
# Важно: Добавляем early_stopping_rounds, который использовался при подборе,
# чтобы потом использовать его при оценке OOF или убрать для финального обучения.
# best_params['early_stopping_rounds'] = 500 # Либо убрать эту строку перед финальным обучением на всех данных


[I 2025-04-05 01:05:26,242] A new study created in memory with name: no-name-176e6879-488b-4ed8-8b8d-38bb99d0194a
[I 2025-04-05 01:08:19,755] Trial 0 finished with value: 27848.124842901587 and parameters: {'iterations': 1037, 'depth': 8, 'learning_rate': 0.02081290525303602, 'l2_leaf_reg': 1.1389644199702988, 'bagging_temperature': 0.4559690629472064, 'subsample': 0.6295096493408531, 'colsample_bylevel': 0.9608216269854182}. Best is trial 0 with value: 27848.124842901587.


Trial 0 finished with Mean CV RMSE: 27848.1248 +/- 6056.7948


[I 2025-04-05 01:11:55,678] Trial 1 finished with value: 27096.902905509975 and parameters: {'iterations': 3920, 'depth': 8, 'learning_rate': 0.0277213807928953, 'l2_leaf_reg': 4.4422510454905195, 'bagging_temperature': 0.3428148178742335, 'subsample': 0.7479900843522618, 'colsample_bylevel': 0.6339947717586244}. Best is trial 1 with value: 27096.902905509975.


Trial 1 finished with Mean CV RMSE: 27096.9029 +/- 5292.3158


[I 2025-04-05 01:14:53,490] Trial 2 finished with value: 27807.622968536125 and parameters: {'iterations': 1844, 'depth': 8, 'learning_rate': 0.04682754211161001, 'l2_leaf_reg': 5.05748664919862, 'bagging_temperature': 0.7332151795635945, 'subsample': 0.8217664737296213, 'colsample_bylevel': 0.9607990789384044}. Best is trial 1 with value: 27096.902905509975.


...

[I 2025-04-05 09:28:28,827] Trial 194 pruned. 
[I 2025-04-05 09:31:01,309] Trial 195 finished with value: 26340.27454711399 and parameters: {'iterations': 1405, 'depth': 5, 'learning_rate': 0.05799429042466648, 'l2_leaf_reg': 2.820058568634953, 'bagging_temperature': 0.8914548672951216, 'subsample': 0.8196099203822671, 'colsample_bylevel': 0.6052930039604614}. Best is trial 184 with value: 25176.782201651506.


Trial 195 finished with Mean CV RMSE: 26340.2745 +/- 5881.4978


[I 2025-04-05 09:33:34,460] Trial 196 pruned. 
[I 2025-04-05 09:36:07,702] Trial 197 finished with value: 25933.621590593375 and parameters: {'iterations': 1795, 'depth': 5, 'learning_rate': 0.05154295209173624, 'l2_leaf_reg': 2.026898618029383, 'bagging_temperature': 0.7441614919854995, 'subsample': 0.8004223706252461, 'colsample_bylevel': 0.9899262750898303}. Best is trial 184 with value: 25176.782201651506.


Trial 197 finished with Mean CV RMSE: 25933.6216 +/- 5533.0764


[I 2025-04-05 09:38:38,979] Trial 198 pruned. 
[I 2025-04-05 09:41:16,518] Trial 199 pruned. 


In [28]:
best_params

{'iterations': 1775,
 'depth': 5,
 'learning_rate': 0.06039828474626542,
 'l2_leaf_reg': 2.132889759843395,
 'bagging_temperature': 0.9470181666458671,
 'subsample': 0.833424506156886,
 'colsample_bylevel': 0.9622754820457864,
 'loss_function': 'RMSE',
 'random_seed': 42,
 'verbose': 0}

# ИТОГИ
Мы не вышли на целевые показатели 12800, что были раньше.
Стало понятно, что логарифмирование целевой переменной и всех фич ухудшает результаты. Так же scaler ни к чему хорошему не приводит. Лучший результат тут был с ансамблем моделей с разными Random State-ами - 13500.